# Resource Scaling Simulation — Horizontal vs Vertical (Colab-ready)

**What this notebook contains**
- A runnable Python simulation that demonstrates **horizontal scaling** (threads) vs **vertical scaling** (processes).
- Ready-to-run in Google Colab: paste the notebook file into Colab or upload it.
- A short **Viva questions & model answers** section you can use during your practical.

**How to run**
1. Open [Google Colab](https://colab.research.google.com).
2. Upload this notebook (File → Upload notebook) or use **File → Open notebook → Upload**.
3. Run the single code cell. It is self-contained and executes the experiments automatically.

**Important notes for Colab**
- The notebook uses `multiprocessing`. Colab runs in a container environment where multiprocessing works, but if a process spawn problem occurs, re-run the cell.
- Adjust the `threads`, `thread_iters`, `processes`, and `proc_base_iters` variables to change experiment intensity.
- If you want live charts or longer runs, ask me and I'll add `psutil` and plotting cells.

---

## Viva Questions (short answers)
1. **What is horizontal scaling?**  
   Adding more instances (scale-out) to distribute load.

2. **What is vertical scaling?**  
   Increasing resources (CPU, RAM) of a single instance (scale-up).

3. **Which Python primitives are used?**  
   `threading.Thread` for horizontal, `multiprocessing.Process` for vertical.

4. **What is GIL?**  
   Global Interpreter Lock in CPython that limits concurrent execution of Python bytecode in threads.

5. **Why use processes for CPU-bound tasks?**  
   Processes bypass the GIL and can run on separate CPU cores.

(There are more viva Qs in the notebook's code comments; review them before the exam.)

*Generated on 2025-11-25 17:59:16*


In [ ]:
# Colab-ready: Horizontal vs Vertical Scaling Simulation
# Run this single cell in Colab to execute the experiments.
# No external pip installs needed.

import time
import threading
import multiprocessing
import os
import math
from datetime import datetime

# ---------- Worker functions (simulate CPU-bound work) ----------
def cpu_workload(iterations, label=None):
    """
    CPU-bound work: calculates many squares to consume CPU.
    iterations: how many loop iterations (larger => more CPU per worker)
    """
    if label:
        print(f"[{datetime.now().strftime('%H:%M:%S')}] {label} started (iter={iterations})")
    s = 0
    for i in range(iterations):
        s += (i * i) % (i + 1 if i else 1)
        # small conditional to avoid heavy I/O
        if i % (iterations // 5 + 1) == 0:
            pass
    if label:
        print(f"[{datetime.now().strftime('%H:%M:%S')}] {label} finished")
    return s

# ---------- Horizontal scaling (threads) ----------
def horizontal_scaling(num_instances=4, iterations_per_instance=250_000):
    """
    Simulate horizontal scaling by spawning multiple threads (each thread = instance).
    """
    print("\n=== Horizontal Scaling (threads) ===")
    print(f"Spawning {num_instances} thread-instances, each with {iterations_per_instance} iterations.")
    threads = []
    start = time.time()
    for i in range(num_instances):
        t = threading.Thread(
            target=cpu_workload,
            args=(iterations_per_instance, f"thread-instance-{i+1}"),
            daemon=False
        )
        threads.append(t)
        t.start()
        time.sleep(0.05)
    for t in threads:
        t.join()
    elapsed = time.time() - start
    print(f"Horizontal scaling (threads) finished in {elapsed:.2f} seconds.\n")
    return elapsed

# ---------- Vertical scaling (processes) ----------
def vertical_scaling(num_cores=2, iterations_base=400_000):
    """
    Simulate vertical scaling by using processes (each process simulates using a CPU core).
    """
    print("\n=== Vertical Scaling (processes) ===")
    print(f"Spawning {num_cores} processes (simulated cores).")
    procs = []
    manager = multiprocessing.Manager()
    return_dict = manager.dict()

    def proc_target(proc_id, iterations, ret_dict):
        res = cpu_workload(iterations, label=f"proc-core-{proc_id}")
        ret_dict[proc_id] = res

    start = time.time()
    for i in range(num_cores):
        iterations = iterations_base * (1 + i//1)
        p = multiprocessing.Process(target=proc_target, args=(i+1, iterations, return_dict))
        procs.append(p)
        p.start()
        time.sleep(0.05)
    for p in procs:
        p.join()
    elapsed = time.time() - start
    print(f"Vertical scaling (processes) finished in {elapsed:.2f} seconds.")
    print(f"Returned results keys (should equal cores): {list(return_dict.keys())}\n")
    return elapsed

# ---------- Combined experiment runner ----------
def run_experiments():
    print("Resource scaling demo - Colab friendly")
    print(f"Python PID: {os.getpid()}, CPU count (os): {os.cpu_count()}\n")
    # Adjustable parameters
    threads = 4
    thread_iters = 250_000
    processes = 2
    proc_base_iters = 400_000

    print(">> Running horizontal scaling experiment (threads)...")
    t_h = horizontal_scaling(num_instances=threads, iterations_per_instance=thread_iters)

    print(">> Running vertical scaling experiment (processes)...")
    t_v = vertical_scaling(num_cores=processes, iterations_base=proc_base_iters)

    print("Summary (lower time = more parallel available for this environment):")
    print(f" - Horizontal (threads) time: {t_h:.2f}s")
    print(f" - Vertical   (processes) time: {t_v:.2f}s")
    if t_h < t_v:
        print("Interpretation hint: In this environment, threads finished faster for these parameters.")
    else:
        print("Interpretation hint: In this environment, processes finished faster for these parameters.")
    print("\nNote: Real cloud scaling behavior depends on actual CPU core count, IO, and scheduler.\n")

# Run experiments when this cell is executed
run_experiments()
